In [4]:
import re
import os
import pandas as pd
import xml.etree.ElementTree as ET


class Limpiar_xml:

    def __init__ (self,nombre_archivo):
        self.nombre_archivo = nombre_archivo

    def lectura_xml(self):

        tree = ET.parse(self.nombre_archivo)
        root = tree.getroot()
        
        self.xml_lista = []

        for child in root:
            xml_dict = {}
            for subchild in child: 
                xml_dict[subchild.tag] = subchild.text
            self.xml_lista.append(xml_dict)
        return self.xml_lista

    def limpieza_xml (self, xml_lista):
        self.xml_lista = xml_lista
        for diccionario in xml_lista:

            if diccionario['gender'] == "0" :
                diccionario["gender"] = "Man"
            elif diccionario["gender"] == "1":
                diccionario["gender"] = "Woman"
            elif diccionario['gender'] == "2":
                diccionario['gender'] = "Nonbinary"
            elif diccionario['gender'] == "3":
                diccionario['gender'] = "Prefer not to say"
            elif diccionario['gender'] == "4":
                diccionario['gender'] = "Prefer to self-describe"
    
        for diccionario in xml_lista:
            diccionario.pop("level_0")

        return self.xml_lista

    def sacar_values(self, xml_lista): 
        self.xml_lista = xml_lista

        values = []
        for i in xml_lista:
            values.append(tuple(i.values()))
        return values

    



In [5]:
xml_clase = Limpiar_xml("data_xml.xml")

In [6]:
fichero_xml = xml_clase.lectura_xml()

In [7]:
xml_limpio = xml_clase.limpieza_xml(fichero_xml)

In [9]:
xml_tuplas = xml_clase.sacar_values(xml_limpio)

In [11]:
def importar_datos (xml_tuplas):
    import mysql.connector
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto1')
    mycursor = cnx.cursor()

    sql = "INSERT INTO tabla_xml (index_xml,time,age,gender) VALUES (%s,%s,%s,%s)"
    val = xml_tuplas
    try:
        mycursor.executemany(sql, val)
        cnx.commit()
        print(mycursor.rowcount, "registro/s insertado/s.")
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [12]:
importar_datos(xml_tuplas)

25972 registro/s insertado/s.
